[DIY Covid-19 Dashboard Kit](https://github.com/fsmeraldi/diy-covid19dash) (C) Fabrizio Smeraldi, 2020 ([f.smeraldi@qmul.ac.uk](mailto:f.smeraldi@qmul.ac.uk) - [web](http://www.eecs.qmul.ac.uk/~fabri/)). All rights reserved.

# DIY Covid-19 Dashboard

This Dashboard displays the UK Government's Covid-19 total PCR tests carried out in relation to the overall planned PCR testing capacity. 

In [276]:
from IPython.display import clear_output
import ipywidgets as wdg
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from uk_covid19 import Cov19API

In [277]:
%matplotlib inline
# make figures larger
plt.rcParams['figure.dpi'] = 100

The data is resourced from the UK Government's Public Health England website https://coronavirus.data.gov.uk. 

In [278]:
# Load JSON files and store the raw data in some variable. Edit as appropriate 

with open("testcomparison.json", "rt") as INFILE:
    info = json.load(INFILE)

In [279]:
def parse_date(datestring):
    """ Convert a date string into a pandas datetime object """
    return pd.to_datetime(datestring, format="%Y-%m-%d")

def wrangle_data(info):
    """ Parameters: rawdata - data from json file or API call. Returns a dataframe.
    Edit to include the code that wrangles the data, creates the dataframe and fills it in. """
    thelist=info['data']
    date=[dictionary['dates'] for dictionary in thelist ]
    date.sort()
    
    startdate=parse_date(date[0])
    enddate=parse_date(date[-1])
    index=pd.date_range(startdate, enddate, freq='D')
    df=pd.DataFrame(index=index, columns=['PCRtestCapacity', 'PCRtotaltests'])
    for entry in thelist:
        dates=parse_date(entry['dates'])
        for column in ['PCRtestCapacity', 'PCRtotaltests']:
            if pd.isna(df.loc[dates,column]):
                value=float(entry[column]) if entry[column]!=None else 0.0
                df.loc[dates, column]=value
    df.fillna(0.0, inplace=True)
    return df


# putting the wrangling code into a function allows you to call it again after refreshing the data through 
# the API. You should call the function directly on the JSON data when the dashboard starts, by including 
# the call in the cell as below:
df=wrangle_data(info) # df is the dataframe for plotting


## Download current data

Refresh below to access current data

In [280]:
 
def access_api(testcomparison):
    """ Accesses the PHE API. Returns raw data in the same format as data loaded from the "canned" JSON file. """
    
    return info # return data read from the API


In [281]:

def api_button_callback(button):
    """ Button callback - it must take the button as its parameter (unused in this case).
    Accesses API, wrangles data, updates global variable df used for plotting. """
    
    apidata=access_api(info)
    # wrangle the data and overwrite the dataframe for plotting
    global df
    df=wrangle_data(apidata)
    # the graph won't refresh until the user interacts with the widget.
    # this function simulates the interaction, see Graph and Analysis below.
    # you can omit this step in the first instance
    
    refresh_graph()
    # after all is done, you can switch the icon on the button to a "check" sign
    # and optionally disable the button - it won't be needed again. You can use icons
    # "unlink" or "times" and change the button text to "Unavailable" in case the 
    # api call fails.
    apibutton.icon="check"
    #apibutton.disabled=True
    
apibutton=wdg.Button(
    description='REFRESH', # you may want to change this...
    disabled=False,
    button_style='INFO', # 'success', 'info', 'warning', 'danger' or ''
    tooltip="Click to refresh",
    # FontAwesome names without the `fa-` prefix - try "download"
    icon='download'
)



apibutton.on_click(api_button_callback) # the name of your function inside these brackets

display(apibutton)



# run all cells before clicking on this button

Button(button_style='info', description='REFRESH', icon='download', style=ButtonStyle(), tooltip='Click to ref…

## Graphs and Analysis

Please find below, the Graph showing the comparison between the PCR Testing Capacity and the total PCR tests done.

In [282]:
#Graph Controls
comparison=wdg.SelectMultiple(
    options=['PCRtestCapacity', 'PCRtotaltests'],
    value=['PCRtestCapacity', 'PCRtotaltests'],
    rows=2,
    description='Statistics:',
    disabled=False
    )
#dropdown interactive widgets    
scale=wdg.Dropdown(
    options=['linear', 'log'],
    description='Scale:',
    disabled=False
    )

controls=wdg.HBox([comparison, scale]) #displayed horizontally

def testcomparison_graph(gcols, gscale):
    plt.show() # important! update won't work properly without this
    if gscale=='linear':
        logscale=False
    else:
        logscale=True
    ncols=len(gcols)
    if ncols>0:
        df[list(gcols)].plot(logy=logscale)
        plt.show() # important - graphs won't update if this is missing 
    else:
        print("Click to select data for graph")
        print("(CTRL-Click to select more than one category)")
        
def refresh_graph():
    """ We change the value of the widget in order to force a redraw of the graph;
    this is useful when the data have been updated. This is a bit of a gimmick; it
    needs to be customised for one of your widgets. """
    current=scale.options
    if current==scale.options[0]:
        other=scale.options[1] 
    else:
        other=scale.options[0]
    scale.options=other # forces the redraw
    scale.options=current
    

graph=wdg.interactive_output(testcomparison_graph, {'gcols': comparison, 'gscale': scale})

display(controls, graph)








Output()

**Author and Copyright Notice** Remember if you deploy this dashboard as a Binder it will be publicly accessible. Take credit for your work! Also acknowledge the data source: *Based on UK Government [data](https://coronavirus.data.gov.uk/) published by [Public Health England](https://www.gov.uk/government/organisations/public-health-england).*